In [13]:
# %%
import os
import yaml
import sys
import json
import time
from tqdm import tqdm
import requests
import pandas as pd

# temporary solution for relative imports when testing the code
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '../..')))
for i in sys.path:
    print(i)
from repo_analytics.utils.generate_query import generate_query

# %%
# Your personal access token from GitHub
with open("../repo_analytics/config/config.yaml", 'r', encoding='utf-8') as file:
    configs = yaml.safe_load(file)
print(configs)
GITHUB_TOKEN_PATH = configs["etl"]["GITHUB_TOKEN_PATH"]
GITHUB_API_URL = configs["etl"]["GITHUB_API_URL"]
GITHUB_TOKEN = json.load(open(GITHUB_TOKEN_PATH, encoding='utf-8'))['key']

# Headers include the authentication token and the content type
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Content-Type": "application/json",
}



/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python312.zip
/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12
/opt/homebrew/Cellar/python@3.12/3.12.3/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload

/Users/shikailiu/.local/share/virtualenvs/repo_analytics-HRaL8j99/lib/python3.12/site-packages
/Users/shikailiu/Desktop/repo_analytics
/Users/shikailiu/Desktop
/Users/shikailiu/Desktop/repo_analytics
/Users/shikailiu/Desktop
/Users/shikailiu/Desktop/repo_analytics
/Users/shikailiu/Desktop
/Users/shikailiu/Desktop/repo_analytics
/Users/shikailiu/Desktop
/Users/shikailiu/Desktop/repo_analytics
/Users/shikailiu/Desktop
{'etl': {'GOOGLE_CREDENTIALS_PATH': '/Users/shikailiu/clean-node-420103-6e83908c3a44.json', 'GITHUB_TOKEN_PATH': '/Users/shikailiu/gh_token.json', 'GITHUB_API_URL': 'https://api.github.com/graphql'}}


In [28]:
# %%
watch_2023 = pd.read_csv("../tmp/2023_watch.csv")
watch_2023['owner'] = watch_2023['name'].str.split('/').str[0]
watch_2023['repo'] = watch_2023['name'].str.split('/').str[1]

query_pairs = list(zip(watch_2023['owner'], watch_2023['repo']))

for i in tqdm(range(1,5000, 200)):
    query = generate_query(query_pairs[i:i+200])

    # Payload is a dictionary with the key 'query' holding the GraphQL query
    payload = {
        "query": query
    }

    # Convert the payload to JSON format
    json_payload = json.dumps(payload)

    # Make the POST request
    response = requests.post(GITHUB_API_URL, headers=headers, data=json_payload)

    # Print the response (status code and returned data)
    #print(response.status_code)
    if response.status_code != 200:
        print("Error: API request failed")
        break
    # file_path = f"./tmp/gh_api_data/data_{i}_{i+99}.json"

    # # Write JSON data to file
    # with open(file_path, "w") as json_file:
    #     json.dump(response.json()['data'], json_file)
    # time.sleep(20)


100%|██████████| 25/25 [01:40<00:00,  4.03s/it]


In [19]:

url = "https://api.github.com/graphql"
token = GITHUB_TOKEN

query = """
{
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}
"""

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

response = requests.post(GITHUB_API_URL, json={'query': query}, headers=headers)

if response.status_code == 200:
    data = response.json()
    rate_limit = data['data']['rateLimit']
    print(f"Limit: {rate_limit['limit']}")
    print(f"Cost: {rate_limit['cost']}")
    print(f"Remaining: {rate_limit['remaining']}")
    print(f"Reset At: {rate_limit['resetAt']}")
else:
    print(f"Query failed with status code: {response.status_code}")
    print(f"Response: {response.json()}")


Limit: 5000
Cost: 1
Remaining: 4980
Reset At: 2024-06-03T07:05:01Z


In [29]:
len(response.json()['data'])

200